In [ ]:
#default_exp reflex_agent

# ReflexAgent

In [ ]:
#export
from collections import defaultdict
import numpy as np
from ecotwins.utility import normalize, distance

class ReflexAgent:
    def __init__(self, eco, n_dir=2):
        self.eco = eco
        self.cur_dir = 0
        self.max_dir = n_dir 

         # Used to determine when to change direction. The direction is changed when
         # we have found/consumed an object.
        self.e_levels = eco.agent.interoception.copy()



    def predict(self, observation, **kwargs):
        # We assume that the first elements of the observation vector is the computed
        # "smell" direction of the different objects. The chosen action is one of these
        # vectors, which one depends on self.cur_dir

        # First check if the direction should change ie an object is consumed.
        self._update_direction()

        action = observation[self.cur_dir * 2: self.cur_dir * 2 + 2]
        if np.isclose(distance(action, 0), 0): # No perception signal, take random step
#             print('Zero observation vector detected')
            action = ecotwins.utility.normalize(np.random.random((2)) - 0.5)
        assert np.isclose((action ** 2).sum(), 1), f'{action}'

        return action, None # None needed in order to mimic stable-baseline3


    def episode(self, n_step=None):
        n_steps = np.iinfo(int).max if n_step is None else n_steps
        obs = self.eco.reset()

        action_dict = defaultdict(int)
        for i in range(n_steps):
            action = obs[self.cur_dir * 2: self.cur_dir * 2 + 2]
            obs, reward, done, _ = self.eco.step(action)

            if done:
                break
            upd = self._update_direction()
            action_dict[upd['obj_name']] += 1

        print(i, done, self.eco.agent.env.agent.interoception, action_dict)

    def _update_direction(self):
        # We do a simple approach here, compute the difference between the current
        # intereception levels and the previous ones. Then loop through all of the
        # homeostatic effects and pick the effect that best explains the interoception
        # change.
        if self.max_dir == 1:
            return # Can't change direction there is only one.
        i_change = (np.array(list(self.e_levels.values())) -
                    np.array(list(self.eco.agent.interoception.values())))
        if (i_change ** 2).sum() == 0: # No action
            return

        # Nothing has happened is currently the best explanation
        best = {'action':None, 'obj_name':None, 'e_value':(i_change ** 2).sum()}
        # print('Initially best explanation', best)
        for (action_type, object_name), v in self.eco.agent.homeostatic_effects.items():
            assert v.keys() == self.eco.agent.interoception.keys()
            dist = i_change + np.array(list(v.values()))
            if (dist ** 2).sum() < best['e_value']:
                best = {'action':action_type, 'obj_name':object_name, 'e_value':(dist ** 2).sum()}
        if best['action'] is not None and best['action'] != 'move':
            # print('Best explanation', best)
            self.cur_dir = ((self.cur_dir + 1) % self.max_dir) 

        self.e_levels = self.eco.agent.interoception.copy()

        return best

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted animal_classes.ipynb.
Converted animation_helper.ipynb.
Converted ecoenv.ipynb.
Converted happiness.ipynb.
Converted index.ipynb.
Converted perception.ipynb.
Converted reflex_agent.ipynb.
Converted tyckande.ipynb.
Converted utility.ipynb.
Converted worlds.ipynb.
